In [ ]:
import numpy as np

from saliencyserieslab.explainers.shap_explainer import ShapExplainer, KernelShapExplainer
from saliencyserieslab.explainers.leftist_explainer import LeftistExplainer
from saliencyserieslab.explainers.lime_explainer import LimeExplainer
from saliencyserieslab.plotting import plot_graph, plot_weighted
from saliencyserieslab.classifier import SktimeClassifier
from saliencyserieslab.load_data import UcrDataset

dataset = "ECG200" # ECG200 SwedishLeaf
modelpath = f"../models/rocket_{dataset}_1"
expname = "LIME"
idx = 99 # 100 / 99


ucr = UcrDataset(
    name=dataset,
    float_dtype=32,
)

test_x, test_y = ucr.load_split("test")
print("test shape : {}".format(test_x.shape))

ts = test_x[idx]

model = SktimeClassifier()

model.load_pretrained_model(modelpath)

if expname == "LEFTIST":

    explainer = LeftistExplainer(
        model=model,
        random_background=test_x,
        perturbation_ratio=0.5,
        num_samples=5_000,
        segment_size=5,
    )

elif expname == "LIME":

    explainer = LimeExplainer(
        model=model,
        perturbation_ratio=0.5,
        num_samples=5_000,
    )

elif expname == "SHAPLEY":

    explainer = ShapExplainer(
        model=model,
        x_background=np.zeros((1,ts.shape[0]))
    )

elif expname == "KERNELSHAP":

    explainer = KernelShapExplainer(
        model=model,
        x_background=np.zeros((1,ts.shape[0]))
    )

w = explainer.explain_instance(ts)

plot_graph(ts, show=True)
plot_weighted(
    ts=ts, 
    w=w,
    modelname=modelpath.split("/")[-1].split("_")[0],
    explainername=expname,
    dataset=dataset,
    show=True,
    colormap="jet",
    )

In [ ]:
import numpy as np

from saliencyserieslab.explainers.shap_explainer import ShapExplainer, KernelShapExplainer
from saliencyserieslab.explainers.leftist_explainer import LeftistExplainer
from saliencyserieslab.explainers.lime_explainer import LimeExplainer
from saliencyserieslab.plotting import plot_graph, plot_weighted
from saliencyserieslab.classifier import SktimeClassifier
from saliencyserieslab.load_data import UcrDataset

dataset = "ECG200" # ECG200 SwedishLeaf
modelpath = f"../models/rocket_{dataset}_1"
expname = "LIME"
idx = 99 # 100 / 99


ucr = UcrDataset(
    name=dataset,
    float_dtype=32,
)

test_x, test_y = ucr.load_split("test")

In [ ]:
from saliencyserieslab.explainers.leftist.LEFTIST import LEFTIST
from saliencyserieslab.explainers.leftist.learning_process.SHAP_learning_process import SHAPLearningProcess
from saliencyserieslab.explainers.leftist.learning_process.LIME_learning_process import LIMELearningProcess
from saliencyserieslab.explainers.leftist.timeseries.segmentator.uniform_segmentator import UniformSegmentator
from saliencyserieslab.explainers.leftist.timeseries.transform_function.mean_transform import MeanTransform
from saliencyserieslab.explainers.leftist.timeseries.transform_function.rand_background_transform import RandBackgroundTransform
from saliencyserieslab.explainers.leftist.timeseries.transform_function.straightline_transform import StraightlineTransform

learning_process_name = "SHAP"
random_state = 42
transform_name = "rand_background"
nb_interpretable_feature = test_x.shape[1]
idx = 55

ts_x = test_x[idx]
ts_y = test_y[idx]

model = SktimeClassifier()

model.load_pretrained_model(modelpath)

if learning_process_name == 'SHAP':
    learning_process = SHAPLearningProcess(
        ts_x,
        model,
        external_dataset=test_x[0].reshape(1,-1),
        )
    
else:
    learning_process = LIMELearningProcess(random_state)


if transform_name == 'mean':
    transform = MeanTransform(ts_x)
elif transform_name == 'straight_line':
    transform = StraightlineTransform(ts_x)
else:
    transform = RandBackgroundTransform(ts_x)
    transform.set_background_dataset(test_x)


segmentator = UniformSegmentator(nb_interpretable_feature)
leftist = LEFTIST(transform,segmentator,model,learning_process)

explanations = leftist.explain(
    1000, 
    ts_x, 
    explanation_size=test_x.shape[1],
    idx_label=ts_y,
    )

for i in range(len(explanations)):
    print('class{} : '.format(i),explanations[i][0])

In [ ]:
from saliencyserieslab.plotting import plot_weighted
import numpy as np

w = np.interp(explanations[0][0], (explanations[0][0].min(), explanations[0][0].max()), (0, 1)).tolist()

plot_weighted(
    ts=ts_x, 
    w=w,
    modelname=modelpath.split("/")[-1].split("_")[0],
    explainername=expname,
    dataset=dataset,
    show=True,
    colormap="jet",
    )

In [ ]:
a = np.array([1, 2, 3]).reshape(1,-1)
a

In [ ]:
c = 30
a = 128 // c

a

In [ ]:
import numpy as np

# Original vector of size 30
original_vector = np.array([0.1, 0.2, 0.3])

# Target size
target_size = 20

# Generate the indices for interpolation
interpolated_vector = np.interp(
    np.linspace(0, len(original_vector) - 1, target_size),
    np.arange(len(original_vector)),
    original_vector
)

print(interpolated_vector)
print(interpolated_vector.shape)

In [ ]:
import pickle
import matplotlib.pyplot as plt

dataset = "Plane"

with open('../data/{}.pkl'.format(dataset), 'rb') as f:
    data = pickle.load(f)[0]


plt.figure(figsize=(6, 3))
for i in range(data.shape[0]):
    plt.plot(data[i], color='blue', alpha=0.5)

# Optional: Add labels, title, and legend
plt.title(dataset)
plt.show()